In [11]:
import numpy as np
from algorithms import steepest_descent, conjugate_gradient, secant, Finite_Difference, armijo
from cost_functions import V_a, gradV_a, V_b, gradV_b, V_1, h1_1, h1_2, V_2, h2_1, h2_2, V_3, h1_3, h2_3
from numpy.linalg import norm, eig
from scipy.optimize import minimize
import time
from sklearn.metrics import mean_squared_error
from decimal import Decimal


In [5]:
x0 = np.array([[0.1],[0.7]])

ineq_cons = {'type': 'ineq',
             'fun' : h1_1}
eq_cons = {'type': 'eq',
           'fun' : h2_1,}

t0 = time.time()
res = minimize(V_1, x0, method='SLSQP', constraints=[eq_cons, ineq_cons])
t = time.time() - t0


xtrue = res.x
i = res.nit
mins = res.fun

rms = mean_squared_error(xtrue, xtrue)



In [6]:
def augmented_lagrangian(x0, 
                         cost_function, 
                         equality_constraints = [], 
                         inequality_constraints = [],
                         threshold = 1e-3, 
                         log = False,
                         track_history = False,):
    '''
    Args: 
        x0 :: np.array
            Initial point of minization. Shape (n,)
        cost_function :: Python function
            Cost function to minimize. Rn -> R. 
        equality_constraints :: list of functions
            List of constraint equalities. 
        inequality constraints :: list of functions
            List of constraint inequalities. 
        threshold :: float
            Threshold at which to stop minimization. Values 
        log :: bool
            True to log optimization progress. Default: False
            Parameter for finite difference estimation. 
        track_history :: bool
            True to track points visited and corresponding cost. 
    Returns: 
        x :: np.array
            Point at which minimization is reached. Shape (n,)
        minimum :: float
            Value of cost function at optimizer. 
        x_history :: list
            List of points visisted. (if track_history = True)
        V_history :: list
            List of costs visisted. (if track_history = True)
    '''
    class P:
        def __init__(self, lambd, sigma):
            self.lambd = lambd
            self.sigma = sigma

        def phi(self, x):
            cost = cost_function(x)
            lambda_eq = lambd[:num_ec , :]
            lambda_ineq = lambd[num_ec:num_c , :]
            sigma_eq = sigma[:num_ec , :]
            sigma_ineq = sigma[num_ec:num_c , :]

            ecs = np.array([ec(x) for ec in equality_constraints])
            cost = cost - sum(lambda_eq * ecs) + 0.5 * sum(sigma_eq * ecs**2)
            
            for i, ineq in enumerate(inequality_constraints):
                ic = ineq(x)
                if ic <= lambda_ineq[i] / sigma_ineq[i]:
                    p_i = np.array([-lambda_ineq[i] * ic + 0.5 * sigma_ineq[i] * ic**2])
                else: 
                    p_i = np.array([-0.5 * lambda_ineq[i]**2 / sigma_ineq[i] ])
                cost = cost + p_i

            return cost

    x_history, V_history = [],[]
    num_ec = len(equality_constraints)
    num_ic = len(inequality_constraints)
    num_c = num_ec + num_ic

    lambd = np.zeros((num_c,1))
    sigma = np.ones((num_c,1))

    c = 1e12 * sigma
    x = x0
    minimum = cost_function(x)
    x_history.append(x), V_history.append(minimum)
    j = 0
    while norm(c) > threshold and all(sigma < 1e12): 
        p = P(lambd, sigma)
        x,_ = steepest_descent(x,
                               p.phi,
                               None,
                               step_size = 'armijo',
                               threshold = 1e-6, 
                               max_iter = 1e4, 
                               fd_method = 'forward')

        previous_cost = c
        inequality_cost = [const(x) for const in inequality_constraints]
        equality_cost = [const(x) for const in equality_constraints]
        c = equality_cost + inequality_cost 

        if norm(c, np.inf) > 0.25 * norm(previous_cost, np.inf):
            for i in range(num_c):
                if np.abs(c[i]) > 0.25 * norm(previous_cost, np.inf):
                    sigma[i] *= 10
            continue
        lambd = lambd - sigma * c

        minimum = cost_function(x).item()
        x_history.append(x), V_history.append(minimum)
        j += 1
        if log:
            print(f'x = {x}, V(x) = {minimum:.5f}')
    if track_history:
        return x, cost_function(x).item(), x_history, V_history
    else:
        return x, cost_function(x).item()

In [7]:
x0 = np.array([[0.1],[0.7]])


cost_function = V_1
inequality_constraints = [h1_1]
equality_constraints = [h2_1]
gradient_function = None

t0 = time.time()
x, minimum, x_hist, v_hist = augmented_lagrangian(x0,
                                            cost_function, 
                                            equality_constraints, 
                                            inequality_constraints,
                                            log = False,
                                            track_history = True)
t = time.time() - t0
rms = mean_squared_error(x, xtrue)


print(f'{x[0].item() :.5f} \t {x[1].item() :.5f} \t {minimum:.5f} \t {Decimal(rms):.2E} \t {t:.3f} \t {len(x_hist)}')

0.71194 	 0.71194 	 1.57612 	 2.33E-5 	 0.115 	 2


In [17]:
##### FUNCTION 2
x0 = np.array([[0.1],[0.7]])
ineq_cons1 = {'type': 'ineq',
             'fun' : h1_2}
ineq_cons2 = {'type': 'ineq',
             'fun' : h2_2,}

t0 = time.time()
res = minimize(V_2, x0, method='SLSQP', constraints=[ineq_cons1, ineq_cons2])
t = time.time() - t0


xtrue = res.x
i = res.nit
mins = res.fun

rms = mean_squared_error(xtrue, xtrue)

print(f'{xtrue[0].item() :.5f} \t {xtrue[1].item() :.5f} \t {mins:.5f} \t {Decimal(rms)} \t {t} \t {i}')

0.66667 	 0.57735 	 -0.38490 	 0 	 0.014425992965698242 	 5


In [23]:
x0 = np.array([[0.6],[0.6]])



cost_function = V_2
inequality_constraints = [h1_2, h2_2]
equality_constraints = []
gradient_function = None

t0 = time.time()
x, minimum, x_hist, v_hist = augmented_lagrangian(x0,
                                            cost_function, 
                                            equality_constraints, 
                                            inequality_constraints,
                                            threshold = 1e-6,
                                            log = False,
                                            track_history = True)
t = time.time() - t0
rms = mean_squared_error(x, xtrue)


print(f'{x[0].item() :.5f} \t {x[1].item() :.5f} \t {minimum:.5f} \t {Decimal(rms):.2E} \t {t:.3f} \t {len(x_hist)}')

0.65360 	 0.57167 	 -0.37364 	 1.01E-4 	 0.140 	 2


In [12]:
##### FUNCTION 3
x0 = np.array([[2],[4]])
ineq_cons1 = {'type': 'ineq',
             'fun' : h1_3}
eq_cons2 = {'type': 'eq',
             'fun' : h2_3,}

t0 = time.time()
res = minimize(V_3, x0, method='SLSQP', constraints=[ineq_cons1, eq_cons2])
t = time.time() - t0


xtrue = res.x
i = res.nit
mins = res.fun

rms = mean_squared_error(xtrue, xtrue)

print(f'{xtrue[0].item() :.5f} \t {xtrue[1].item() :.5f} \t {mins:.5f} \t {Decimal(rms)} \t {t} \t {i}')

1.00000 	 1.73205 	 -0.73205 	 0 	 0.025768041610717773 	 6


In [16]:
x0 = np.array([[0.1],[0.7]])


cost_function = V_3
inequality_constraints = [h1_3]
equality_constraints = [h2_3]
gradient_function = None

t0 = time.time()
x, minimum, x_hist, v_hist = augmented_lagrangian(x0,
                                            cost_function, 
                                            equality_constraints, 
                                            inequality_constraints,
                                            threshold = 1e-4,
                                            log = False,
                                            track_history = True)
t = time.time() - t0
rms = mean_squared_error(x, xtrue)


print(f'{x[0].item() :.5f} \t {x[1].item() :.5f} \t {minimum:.5f} \t {Decimal(rms):.2E} \t {t:.3f} \t {len(x_hist)}')

0.99999 	 1.73206 	 -0.73209 	 9.55E-11 	 0.402 	 6
